In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
622,Excellent performance. There still are good ac...,positive
227,Greetings again from the darkness. Director Al...,positive
430,I love Jane Austen's stories. I've only read t...,negative
6,This is a good example a film that in spite of...,positive
738,Saw this on SBS TV here in Australia the other...,positive


In [4]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [5]:
df = normalize_text(df)
df.head()

,review,sentiment
622,excellent performance still good actor around ...,positive
227,greeting darkness director alejandro amenabar ...,positive
430,love jane austen s story read two p p s s seen...,negative
6,good example film spite low rating worth watch...,positive
738,saw sb tv australia week titled laputa castle ...,positive


In [6]:
df['sentiment'].value_counts()

sentiment
negative    264
positive    236
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
622,excellent performance still good actor around ...,1
227,greeting darkness director alejandro amenabar ...,1
430,love jane austen s story read two p p s s seen...,0
6,good example film spite low rating worth watch...,1
738,saw sb tv australia week titled laputa castle ...,1


In [9]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [15]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [17]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/rahul-nayak01/ML-OPs-Model-deployment-Mini-project-.mlflow')
dagshub.init(repo_owner='rahul-nayak01', repo_name='ML-OPs-Model-deployment-Mini-project-', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Initialized MLflow to track repo "rahul-nayak01/ML-OPs-Model-deployment-Mini-project-"

2025-11-09 10:41:14,659 - INFO - Initialized MLflow to track repo "rahul-nayak01/ML-OPs-Model-deployment-Mini-project-"


Repository rahul-nayak01/ML-OPs-Model-deployment-Mini-project- initialized!

2025-11-09 10:41:14,663 - INFO - Repository rahul-nayak01/ML-OPs-Model-deployment-Mini-project- initialized!


<Experiment: artifact_location='mlflow-artifacts:/20f36992cb3645bd962fa87286cfd420', creation_time=1762664683677, experiment_id='0', last_update_time=1762664683677, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [18]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-11-09 10:41:17,587 - INFO - Starting MLflow run...
2025-11-09 10:41:18,465 - INFO - Logging preprocessing parameters...
2025-11-09 10:41:19,496 - INFO - Initializing Logistic Regression model...
2025-11-09 10:41:19,499 - INFO - Fitting the model...
2025-11-09 10:41:19,515 - INFO - Model training complete.
2025-11-09 10:41:19,515 - INFO - Logging model parameters...
2025-11-09 10:41:19,909 - INFO - Making predictions...
2025-11-09 10:41:19,910 - INFO - Calculating evaluation metrics...
2025-11-09 10:41:19,921 - INFO - Logging evaluation metrics...
2025-11-09 10:41:21,467 - INFO - Saving and logging the model...
2025/11/09 10:41:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-11-09 10:41:22,689 - ERROR - An error occurred: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
Traceback (most recent call last):
  File "C:\Users\nayak\AppData\Local\Temp\ipykernel_18704\2979654972.py", line 48, in 

🏃 View run bouncy-swan-62 at: https://dagshub.com/rahul-nayak01/ML-OPs-Model-deployment-Mini-project-.mlflow/#/experiments/0/runs/e789b0414726450d8c73b7e2b94d1460
🧪 View experiment at: https://dagshub.com/rahul-nayak01/ML-OPs-Model-deployment-Mini-project-.mlflow/#/experiments/0
